In [1]:
## importing necessary Libraries 
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import PIL 
from PIL import Image
import cv2 as cv
import os

In [2]:
data='E:/data science work/data/face mask dataset'

In [4]:
## importing tenserflow libraries for making deeplearning Model
from keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array
from keras.layers import Dense,Flatten,Conv2D,MaxPooling2D,Dropout
from keras.models import Model,Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Activation




# just trying something else methods for taking the data



In [5]:
# label={0:'mask',1:'no mask'}
# face_classifier=cv.CascadeClassifier('Haarcascades/haarcascade_frontalface_default.xml')

In [31]:
# data=[]
# target=[]
# for folder in os.listdir('face mask dataset'):
#     for i in os.listdir('face mask dataset/{}'.format(folder)):
#         image=cv.imread('face mask dataset/{}/{}'.format(folder,i))
#         gray=cv.cvtColor(image,cv.COLOR_BGR2GRAY)
#         faces=face_classifier.detectMultiScale(gray,1.2,5)
#         if (faces!=()):
#             for a,b,c,d in faces:
#                 crop=gray[b:b+d,a:a+c]
#                 if crop!=():
#                     resized=cv.resize(crop,(224,224))
#                     data.append(resized)
#                     target.append(folder)
#         else:
#             continue
            
            
        

# upto this

In [5]:
train_datagen=ImageDataGenerator(rescale=1/255,horizontal_flip=True,shear_range=.2,zoom_range=.2)
test_datagen=ImageDataGenerator(rescale=1/255)

In [6]:
train_generator=train_datagen.flow_from_directory(data,target_size=(224,224),class_mode='categorical',shuffle=True,batch_size=25)


Found 1376 images belonging to 2 classes.


In [7]:
train_generator.class_indices

{'with mask': 0, 'without mask': 1}

In [54]:
## the dim array RGB
model=Sequential()
model.add(Conv2D(32,kernel_size=3,input_shape=(224,224,3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(32,kernel_size=3,activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dense(100,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2,activation='softmax'))
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [36]:
# ## the 1dim array GRAY
# model=Sequential()
# model.add(Conv2D(32,kernel_size=3,input_shape=(224,224,1),activation='relu'))
# model.add(MaxPooling2D(2,2))
# model.add(Conv2D(32,kernel_size=3,activation='relu'))
# model.add(MaxPooling2D(2,2))
# model.add(Flatten())
# model.add(Dense(100,activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(1,activation='sigmoid'))
# model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])


In [37]:
# deep model
# model=Sequential()

# model.add(Conv2D(200,(3,3),input_shape=(224,224,3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))
# #The first CNN layer followed by Relu and MaxPooling layers

# model.add(Conv2D(100,(3,3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))
# #The second convolution layer followed by Relu and MaxPooling layers

# model.add(Flatten())
# model.add(Dropout(0.5))
# #Flatten layer to stack the output convolutions from second convolution layer
# model.add(Dense(50,activation='relu'))
# #Dense layer of 64 neurons
# model.add(Dense(2,activation='softmax'))
# #The Final layer with two outputs for two categories

# model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [8]:
#transfer learning
vgg=VGG16(input_shape=(224,224,3),weights='imagenet',include_top=False)
for layer in vgg.layers:
    layer.trainable=False
x=Flatten()(vgg.output)
x=Dense(50,activation='relu')(x)
final=Dense(2,activation='softmax')(x)
model1=Model(inputs=vgg.input,outputs=final)
model1.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [39]:
# for non generator data
# model.fit(array,target,epochs=10,batch_size=20,validation_split=.2)

In [9]:
model1.fit_generator(train_generator,epochs=10,steps_per_epoch=len(train_generator))

Instructions for updating:
Use tf.cast instead.
Epoch 1/3
56/56 [==============================] - 842s 15s/step - loss: 0.6073 - acc: 0.8428
Epoch 2/3
56/56 [==============================] - 835s 15s/step - loss: 0.0510 - acc: 0.9914
Epoch 3/3
56/56 [==============================] - 19524s 349s/step - loss: 0.0337 - acc: 0.9907


In [41]:
# k=model1.predict_generator(train_generator,steps=20)

In [10]:
label={0:'mask',1:'no mask'}
face_classifier=cv.CascadeClassifier('Haarcascades/haarcascade_frontalface_default.xml')

In [17]:
import cv2 as cv
def detects(frame):
    gray=cv.cvtColor(frame,cv.COLOR_BGR2GRAY)
    faces=face_classifier.detectMultiScale(gray,1.2)
    for a,b,c,d in faces:
        crop=frame[b:b+d,a:a+c]
        resized=cv.resize(crop,(224,224))
        resized=resized/255.0
        resized=np.expand_dims(resized,axis=0)
        pred=model1.predict(resized)
        prediction=np.argmax(pred,axis=1)[0]
        pct=str(int(np.max(pred,axis=1)[0]*100))
        if prediction==0:
            cv.rectangle(frame,(a,b),(a+c,b+d),(0,255,0),3)
            cv.putText(frame,label[0]+pct+'%',(a,b-10),cv.FONT_HERSHEY_SIMPLEX,.8,(0,255,0),2)
        else:
            cv.rectangle(frame,(a,b),(a+c,b+d),(0,0,255),3)
            cv.putText(frame,label[1]+pct+'%',(a,b-10),cv.FONT_HERSHEY_SIMPLEX,.8,(0,0,255),2)
    return frame
            

In [18]:
c=cv.VideoCapture(0)
while True:
    ret,frame=c.read()
    cv.imshow('live mask prediction',detects(frame))
    if cv.waitKey(1)==13:
        break
c.release()
cv.destroyAllWindows()
    
    
    

# hosting the appte server

In [22]:
cv.destroyAllWindows()

In [23]:
import cv2 as cv
from flask import Flask,render_template 

In [ ]:
# BASE.HTML
# <html lang="en">

# <head>
#     <meta charset="UTF-8">
#     <meta name="viewport" content="width=device-width, initial-scale=1.0">
#     <meta http-equiv="X-UA-Compatible" content="ie=edge">
#     <title>AI Demo</title>
#     <link href="https://cdn.bootcss.com/bootstrap/4.0.0/css/bootstrap.min.css" rel="stylesheet">
#     <script src="https://cdn.bootcss.com/popper.js/1.12.9/umd/popper.min.js"></script>
#     <script src="https://cdn.bootcss.com/jquery/3.3.1/jquery.min.js"></script>
#     <script src="https://cdn.bootcss.com/bootstrap/4.0.0/js/bootstrap.min.js"></script>
#     <link href="{{ url_for('static', filename='css/main.css') }}" rel="stylesheet">      
# </head>

# <body>
#     <nav class="navbar navbar-dark bg-dark">
#         <div class="container">
#             <a class="navbar-brand" href='{{url_for('pre')}}'>AI Demo</a>
#             <button class="btn btn-outline-secondary my-2 my-sm-0" type="submit">Help</button>
#         </div>
#     </nav>
#     <div class="container">
# <a href='{{url_for('pre')}}'> start face detection</a>
#         <div id="content" style="margin-top:2em">{% block content %}{% endblock %}</div>
#     </div>


# </body>

# </html>
#  INDEX.HTML
# {% extends "base.html" %} {% block content %}

# <h2>Image Classifier</h2>

# <div>
#     <form id="upload-file" method="post" enctype="multipart/form-data">
#         <label for="imageUpload" class="upload-label">
#             Choose...
#         </label>
#         <input type="file" name="file" id="imageUpload" accept=".png, .jpg, .jpeg">
#     </form>

#     <div class="image-section" style="display:none;">
#         <div class="img-preview">
#             <div id="imagePreview">
#             </div>
#         </div>
#         <div>
#             <button type="button" class="btn btn-primary btn-lg " id="btn-predict">Predict!</button>
#         </div>
#     </div>

#     <div class="loader" style="display:none;"></div>

#     <h3 id="result">
#         <span> </span>
#     </h3>

# </div>

# {% endblock %}

In [29]:
def detect1(frame):
    gray=cv.cvtColor(frame,cv.COLOR_BGR2GRAY)
    faces=face_classifier.detectMultiScale(gray,1.2)
    for a,b,c,d in faces:
        crop=frame[b:b+d,a:a+c]
        resized=cv.resize(crop,(224,224))
        resized=resized/255.0
        resized=np.expand_dims(resized,axis=0)
        pred=model1.predict(resized)
        prediction=np.argmax(pred,axis=1)[0]
        pct=str(int(np.max(pred,axis=1)[0]*100))
        if prediction==0:
            cv.rectangle(frame,(a,b),(a+c,b+d),(0,255,0),3)
            cv.putText(frame,label[0]+pct+'%',(a,b-10),cv.FONT_HERSHEY_SIMPLEX,.8,(0,255,0),2)
        else:
            cv.rectangle(frame,(a,b),(a+c,b+d),(0,0,255),3)
            cv.putText(frame,label[1]+pct+'%',(a,b-10),cv.FONT_HERSHEY_SIMPLEX,.8,(0,0,255),2)
    return frame
            
app=Flask(__name__,template_folder='template')
@app.route('/')
def index():
    return render_template('base.html')
@app.route('/prediction',methods=['POST','GET'])
def pre():
    c=cv.VideoCapture(0)
    while True:
        ret,frame=c.read()
        cv.imshow('live mask prediction',detect1(frame))
        if cv.waitKey(1)==13:
            break        
    c.release()
    cv.destroyAllWindows()
    return render_template('index.html')
    
    
    
    
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Oct/2020 03:40:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Oct/2020 03:40:46] "GET /prediction HTTP/1.1" 200 -
